In [50]:
import random
from math import log2, ceil
from decimal import Decimal
import benchmark_functions as bf
import scipy

class Chromosom:
    def __init__(self, precision, variables_count, start_, end_):
        self.precision = precision
        self.variables_count = variables_count
        self.start_ = start_
        self.end_ = end_
        self.chromosom_len = ceil(self.precision * log2(self.end_ - self.start_))
        self.chromosoms = self._generate_chromosom()
        self.decoded_chromosom = self._decode_chromosom()

    def _generate_chromosom(self) -> list:
      chromosoms = []
      for i in range(self.variables_count):
        chromosom = []
        for i in range(self.chromosom_len):
            chromosom.append(random.randint(0, 1))
        chromosoms.append(chromosom)
      return chromosoms

    def _decode_chromosom(self) -> list:
      decoded_chromosom = []
      for chromosom in self.chromosoms:
        decimal_number = sum(bit * (2 ** i) for i, bit in enumerate(reversed(chromosom)))
        decoded = self.start_ + decimal_number * (self.end_ - self.start_) / (2 ** self.chromosom_len - 1)
        decoded_chromosom.append(Decimal(decoded).quantize(Decimal(10) ** -self.precision))
      return decoded_chromosom
    
    def __str__(self):
        return f"Chromosoms: {self.chromosoms} | Value in Decimal: {self.decoded_chromosom}"


class Individual:
    def __init__(self, precision, variables_count, start_, end_):
        self.chromosom = Chromosom(precision, variables_count, start_, end_)
        self.variables_count = variables_count

    def __str__(self):
        return f"{self.chromosom}"


class Population:
    def __init__(self, variables_count, population_size, precision, start_, end_):
        self.variables_count = variables_count
        self.population_size = population_size
        self.individuals = [Individual(precision, variables_count, start_, end_) for _ in range(self.population_size)]

    def __call__(self, func: any):
        cell_values = []
        X = self.getX()
        for x in X:
            cell_value = func(x)
            cell_values.append(cell_value)
        return cell_values
    
    def getX(self) -> list:
        X = []
        for individual in self.individuals:
            values = individual.chromosom.decoded_chromosom
            values = [float(x) for x in values]
            X.append(values)
        return X

    def __str__(self):
        return "\n".join(str(individual) for individual in self.individuals)


In [49]:
c = Chromosom(10, 5, -10, 10)
chromosoms = c.chromosoms
decoded_chromosom = c.decoded_chromosom
print(c)


Chromosoms: [[0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0], [1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0], [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0], [1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0]] | Value in Decimal: [Decimal('-5.2081183495'), Decimal('0.5547402252'), Decimal('-8.9355203786'), Decimal('-7.1839908996'), Decimal('7.2059088036')]


In [53]:

population = Population(5, 5, 10, -10, 10)

func = bf.Schwefel(n_dimensions=5)

X = population.getX()
values = population(func)
print("X: ", X)
print("Values: ", values)



X:  [[0.8492680481, -3.3891990317, -7.7753572923, 6.6147280565, 2.2560199554], [1.5858029638, -3.7142881957, -4.9872697199, -5.3391176929, -6.9035428087], [8.3800335088, -2.4441679657, 1.5618265491, -3.7799198056, 4.1254632399], [5.5397362691, -0.3917640367, -9.1573717465, -0.6147242978, 4.1060017929], [-5.1772240659, 9.1401908713, -9.9508639116, 1.0032140492, -6.6486224594]]
Values:  [2094.3750588511957, 2108.1566248423765, 2093.6530327151413, 2089.018833029918, 2100.355733798105]
